In [ ]:
# Import necessary libraries
import pandas as pd
import yfinance as yf
import pandas_datareader.data as pdr
from datetime import datetime

In [ ]:
# Constants
START_DATE = '1990-01-01'  # Start date for fetching data
END_DATE = '2022-01-01'    # End date for fetching data
EXPORT_PATH = '/home/student/suahmad/Richter_Final/Main_files/'  # Directory to save the output CSV files

In [ ]:
# Function to fetch macroeconomic data from FRED
def fetch_macro_data():
    # GDP data
    gdp = pdr.DataReader('GDP', 'fred', START_DATE, END_DATE)
    
    # Unemployment rate data
    unemployment = pdr.DataReader('UNRATE', 'fred', START_DATE, END_DATE)
    unemployment.rename(columns={'UNRATE': 'Unemployment_Rate'}, inplace=True)
    
    # USD to Euro exchange rate
    exchange = pdr.DataReader('DEXUSEU', 'fred', START_DATE, END_DATE)
    exchange.rename(columns={'DEXUSEU': 'Exchange_Rate(USD to Euro)'}, inplace=True)
    
    # Federal funds rate
    fed_funds = pdr.DataReader('FEDFUNDS', 'fred', START_DATE, END_DATE)
    fed_funds.rename(columns={'FEDFUNDS': 'Federal_Funds_Rate'}, inplace=True)

    # Combine all macroeconomic indicators into one DataFrame
    data = gdp.join([unemployment, exchange, fed_funds], how='outer')
    data.reset_index(inplace=True)
    data['DATE'] = pd.to_datetime(data['DATE']).dt.tz_localize(None)  # Remove timezone info

    # Forward fill missing values for critical indicators
    data['GDP'] = data['GDP'].ffill()
    data['Federal_Funds_Rate'] = data['Federal_Funds_Rate'].ffill()
    data['Unemployment_Rate'] = data['Unemployment_Rate'].ffill()

    return data

# Function to fetch VIX (volatility index) data using yfinance
def fetch_vix_data():
    ticker = yf.Ticker('^VIX')
    vix = ticker.history(start=START_DATE, end=END_DATE)[['Close']]
    vix.reset_index(inplace=True)
    vix.rename(columns={'Date': 'DATE', 'Close': 'VIX'}, inplace=True)
    vix['DATE'] = pd.to_datetime(vix['DATE']).dt.tz_localize(None)  # Remove timezone info
    return vix

# Function to fetch historical stock data for a given ticker symbol
def fetch_stock_data(ticker_symbol):
    ticker = yf.Ticker(ticker_symbol)
    df = ticker.history(start=START_DATE, end=END_DATE)[['Close']]
    df.reset_index(inplace=True)
    df.rename(columns={'Date': 'DATE', 'Close': 'Close_Price'}, inplace=True)
    df['DATE'] = pd.to_datetime(df['DATE']).dt.tz_localize(None)  # Remove timezone info
    return df

# Function to compile macro, VIX, and stock data into a single DataFrame
def compile_data(ticker_symbol):
    macro = fetch_macro_data()
    vix = fetch_vix_data()
    stock = fetch_stock_data(ticker_symbol)

    # Merge datasets on DATE
    df = macro.merge(vix, on='DATE', how='outer')
    df = df.merge(stock, on='DATE', how='outer')
    return df

# Function to remove rows with any missing (NA) values
def remove_na_values(df):
    df_cleaned = df.dropna()
    return df_cleaned

# Function to compile data for a stock ticker and save it as a CSV file
def save_data_for_ticker(ticker_symbol):
    df = compile_data(ticker_symbol)
    df = remove_na_values(df)
    file_path = f"{EXPORT_PATH}{ticker_symbol}_combined.csv"
    df.to_csv(file_path, index=False)
    print(f"✅ File saved to {file_path}")

In [ ]:
# Run for all tickers
tickers = ['LLY', 'UNH', 'JNJ','ABBV','ABT','MRK','TMO','ISRG','GILD','CVS','CI','BIIB','REGN','VRTX','DHR','MCK','BMY','PFE','AMGN']

for ticker in tickers:
    save_data_for_ticker(ticker)